In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import logging
import pickle
from joblib import Parallel, delayed
import gc
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [2]:
base_path = './data'
feature_path = './feature'

In [ ]:
# 话题向量
# topic_table = pd.read_csv(f'{base_path}/topic_vectors_64d.txt', sep='\t', header=None)
# topic_table.columns = ['topic', 'vec']

# def str2vec(s):
#     tmp = s.split(' ')
#     res = []
#     for num in tmp:
#         res.append(float(num))
#     return res

# topic_table['vec'] = topic_table['vec'].apply(str2vec)

# import pickle
# pickle.dump(topic_table, open(f'{base_path}/topic_vec.pkl', 'wb'))

In [3]:
topic_table = pickle.load(open(f'{base_path}/topic_vec.pkl', 'rb'))

In [4]:
# 问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t', usecols=[0, 6])
ques.columns = ['qid', 'topic']

logging.info("ques %s", ques.shape)

[2019-12-17 04:26:56,821] INFO in <ipython-input-4-26315363bfb1>: ques (1829900, 2)


In [ ]:
# 回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t', usecols=[0, 1, 2, 3])
ans.columns = ['aid', 'qid', 'uid', 'ans_dt']
logging.info("ans %s", ans.shape)

In [ ]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))

ans['day'] = extract_day(ans['ans_dt'])
del ans['ans_dt']
logging.info("ans %s", ans.shape)

In [ ]:
# ans 对应的问题的话题
ans = pd.merge(ans, ques, on='qid', how='left')

In [ ]:
# 4折统计
def fold_fn(x):
    if 3838<=x<=3846:
        return 0
    if 3847<=x<=3853:
        return 1
    if 3854<=x<=3860:
        return 2
    if 3861<=x<=3867:
        return 3
    else:
        return -1     # 更前的一个月

In [ ]:
# train
ans['fold'] = ans['day'].apply(fold_fn)

In [ ]:
ans_with_topic = ans[ans['topic']!='-1']

In [ ]:
user_topic_0 = ans_with_topic[ans_with_topic['fold']!=0].groupby('uid')['topic'].agg(','.join)
logging.info('fold 0, user num: %s', len(user_topic_0))

user_topic_1 = ans_with_topic[ans_with_topic['fold']!=1].groupby('uid')['topic'].agg(','.join)
logging.info('fold 1, user num: %s', len(user_topic_1))

user_topic_2 = ans_with_topic[ans_with_topic['fold']!=2].groupby('uid')['topic'].agg(','.join)
logging.info('fold 2, user num: %s', len(user_topic_2))

user_topic_3 = ans_with_topic[ans_with_topic['fold']!=3].groupby('uid')['topic'].agg(','.join)
logging.info('fold 3, user num: %s', len(user_topic_3))

In [ ]:
# test
# user_topic_all = ans.groupby('uid')['topic'].agg(','.join)  # 给 test 用
# pickle.dump(user_topic_all, open(f'{base_path}/user_topic.pkl', 'wb'))

In [5]:
user_topic_all = pickle.load(open(f'{base_path}/user_topic.pkl', 'rb'))

In [6]:
# 压缩数据
def compress_data(df):
    t = df.dtypes
    for x in t[t == 'int64'].index:
        df[x] = df[x].astype('int32')

    for x in t[t == 'float64'].index:
        df[x] = df[x].astype('float32')
    return df

In [7]:
def cos_sim(vec1, vec2):
    return np.dot(vec1, vec2)/((np.linalg.norm(vec1)*(np.linalg.norm(vec2))))

def eucl_sim(vec1, vec2):
    return np.linalg.norm(np.array(vec1)-np.array(vec2))

# feature 1

In [7]:
# 用户在其他时期回答的话题统计特征
def user_topic_stat(u_topic):
    n_most_common = 20     # 截取回答次数前面的话题
        
    if u_topic == '-1':
        return ([0] + [np.nan]*15)
    
    u_topic_2 = u_topic.split(',')
    counter = Counter(u_topic_2)          
    most_common_topic = counter.most_common(n_most_common)
    
    u_topic_most_com = []     # 前几个最常回答的话题
    count_list = []           # 这几个常回答的话题对应的计数
    for ut_c in most_common_topic: 
        u_topic_most_com.append(ut_c[0])
        count_list.append(ut_c[1])
    count_norm_list = np.array(list(count_list)) / np.sum(list(count_list))   # 这几个常回答话题对应的计数比例
    
    """
    用户话题的统计
    """
    # count_list 的统计信息
    count_u_topic = len(count_list)  
    mean_u_topic = np.mean(count_list)
    std_u_topic = np.std(count_list)  
    sum_u_topic = np.sum(count_list)
    max_u_topic = np.max(count_list)
    min_u_topic = np.min(count_list)
    # res 1
    res = [count_u_topic, mean_u_topic, std_u_topic, sum_u_topic, max_u_topic, min_u_topic]
    
    if len(most_common_topic)>1:   # 2 以上，计算两两相似度
        sim_list = []
        sim_list_eucl = []
        for i in range(len(most_common_topic)):
            for j in range(i+1, len(most_common_topic)):
                ut1_index = int(u_topic_most_com[i][1:]) - 1
                ut2_index = int(u_topic_most_com[j][1:]) - 1
                ut1 = topic_table.iloc[ut1_index]['vec']
                ut2 = topic_table.iloc[ut2_index]['vec']

                sim_list.append(cos_sim(ut1, ut2))
                sim_list_eucl.append(eucl_sim(ut1, ut2))
        # 余弦相似度
        min_uu_sim = np.min(sim_list)
        max_uu_sim = np.max(sim_list)
        sum_uu_sim = np.sum(sim_list)
        mean_uu_sim = np.mean(sim_list)
        std_uu_sim = np.std(sim_list)
        # 欧氏距离
        min_uu_sim_eucl = np.min(sim_list_eucl)
        max_uu_sim_eucl = np.max(sim_list_eucl)
        sum_uu_sim_eucl = np.sum(sim_list_eucl)
        mean_uu_sim_eucl = np.mean(sim_list_eucl)
        std_uu_sim_eucl = np.std(sim_list_eucl)
        # res 2
        res += [min_uu_sim, max_uu_sim, sum_uu_sim, mean_uu_sim, std_uu_sim]
        res += [min_uu_sim_eucl, max_uu_sim_eucl, sum_uu_sim_eucl, mean_uu_sim_eucl, std_uu_sim_eucl]
    else:       # 用户回答个数为 1
        # res 2
        res += [np.nan] * 10
    
    return res

In [ ]:
import traceback

processes = 36   # 设置进程数

def target_fn(series, path, num):
    try:
        logging.info('start to extract feature.')
        df_from_series = series.reset_index()
        t1 = df_from_series.apply(lambda x: user_topic_stat(x['topic']),axis=1, result_type='expand')
        col_name = ['count_u_topic', 'mean_u_topic', 'std_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic', 
                    'min_uu_sim', 'max_uu_sim', 'sum_uu_sim', 'mean_uu_sim', 'std_uu_sim',
                    'min_uu_sim_eucl', 'max_uu_sim_eucl', 'sum_uu_sim_eucl', 'mean_uu_sim_eucl', 'std_uu_sim_eucl']
        t1.columns = col_name
        res = pd.concat([df_from_series, t1], axis=1)
        logging.info('extracting finish.')
        
        # 压缩数据
        res = compress_data(res)
        res.to_csv(f'{path}/user_topic_feature_{num}.txt', index=False, sep='\t')
        logging.info('file %s saving finish.', num)
        del res
        gc.collect()
    except:
        print(traceback.print_exc())
        
def multi_proc_1(series, frag_path):
    import multiprocessing
    pool = multiprocessing.Pool(processes=processes)
    len_data = len(series)
    len_batch = len_data // processes
    for i in range(processes):
        start = i * len_batch
        end = (i+1) * len_batch
        if i == (processes-1):
            end = len_data
        tmp = series[start:end]
        pool.apply_async(target_fn, (tmp, frag_path, i))
        
    pool.close()
    pool.join()

In [ ]:
multi_proc_1(user_topic_0, './temp_user_topic_stat/fold0')
multi_proc_1(user_topic_1, './temp_user_topic_stat/fold1')
multi_proc_1(user_topic_2, './temp_user_topic_stat/fold2')
multi_proc_1(user_topic_3, './temp_user_topic_stat/fold3')

In [ ]:
# 合数据
def concat_data(original_path, dest_path):
    topic_feat = None
    for i in range(processes):
        d = pd.read_csv(f'{original_path}/user_topic_feature_{i}.txt', sep='\t')
        if topic_feat is None:
            topic_feat = d
        else:
            topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
            
    logging.info('topic feature, shape: %s, start saving', topic_feat.shape)
    
    topic_feat = compress_data(topic_feat)
    topic_feat.to_csv(f'{dest_path}/topic_feature.txt', index=False, sep='\t')
    logging.info('saving done. From %s to %s', original_path, dest_path)

In [ ]:
concat_data('./temp_user_topic_stat/fold0', f'./temp_user_topic_stat/dest0')
concat_data('./temp_user_topic_stat/fold1', f'./temp_user_topic_stat/dest1')
concat_data('./temp_user_topic_stat/fold2', f'./temp_user_topic_stat/dest2')
concat_data('./temp_user_topic_stat/fold3', f'./temp_user_topic_stat/dest3')

In [ ]:
pickle.dump(user_topic_0, open(f'{base_path}/user_topic_fold_0.pkl', 'wb'))
pickle.dump(user_topic_1, open(f'{base_path}/user_topic_fold_1.pkl', 'wb'))
pickle.dump(user_topic_2, open(f'{base_path}/user_topic_fold_2.pkl', 'wb'))
pickle.dump(user_topic_3, open(f'{base_path}/user_topic_fold_3.pkl', 'wb'))

In [ ]:
ut_stat_0 = pd.read_csv('./temp_user_topic_stat/dest0/topic_feature.txt', sep='\t')
ut_stat_1 = pd.read_csv('./temp_user_topic_stat/dest1/topic_feature.txt', sep='\t')
ut_stat_2 = pd.read_csv('./temp_user_topic_stat/dest2/topic_feature.txt', sep='\t')
ut_stat_3 = pd.read_csv('./temp_user_topic_stat/dest3/topic_feature.txt', sep='\t')

In [ ]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
train['day'] = extract_day(train['dt'])
del train['dt']
logging.info("train %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
del test['dt']
logging.info("test %s", test.shape)

In [ ]:
train = pd.merge(train, ques, on='qid', how='left')
test = pd.merge(test, ques, on='qid', how='left')

In [ ]:
# train
train['fold'] = train['day'].apply(fold_fn)

train['user_topic_kfold'] = -10000

for i in range(4):
    train.loc[train['fold']==i, 'user_topic_kfold'] = train[train['fold']==i]['uid'].map(ut_list[i]).fillna('-1')

assert len(train[train['user_topic_kfold']==-10000])==0

In [ ]:
# test
user_topic = pickle.load(open(f'{base_path}/user_topic.pkl', 'rb'))       # 没有打折
test['user_topic_kfold'] = test['uid'].map(user_topic).fillna('-1')    # 实际并不是 kfold

In [ ]:
# 过去两个月的用户话题特征，给 test 数据用 
multi_proc_1(user_topic, './temp_user_topic_stat/all')
# 合数据
concat_data('./temp_user_topic_stat/all', f'./temp_user_topic_stat/all_dest')

In [ ]:
# train, 用户在其他时期回答的话题特征 merge 回来
ut_stat_list = [ut_stat_0, ut_stat_1, ut_stat_2, ut_stat_3]
ut_stat_feat = ['count_u_topic', 'mean_u_topic', 'std_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic', 
                'min_uu_sim', 'max_uu_sim','sum_uu_sim', 'mean_uu_sim', 'std_uu_sim', 
                'min_uu_sim_eucl', 'max_uu_sim_eucl', 'sum_uu_sim_eucl', 'mean_uu_sim_eucl', 'std_uu_sim_eucl']
for f in ut_stat_feat:
    train[f] = -10000
for i in range(4):
    train.loc[train['fold']==i, ut_stat_feat] = pd.merge(train.loc[train['fold']==i]['uid'], ut_stat_list[i], 
                                                         on='uid', how='left')[ut_stat_feat].values
for f in ut_stat_feat:
    assert len(train[train[f]==-10000]) == 0

# 压缩数据
int_feat = ['count_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic']
train[int_feat] = train[int_feat].fillna(0).astype('int32')
for f in ut_stat_feat:
    if f not in int_feat:
        train[f] = train[f].astype('float32')

In [ ]:
# test, 用户在其他时期回答的话题特征 merge 回来
ut_stat_all = pd.read_csv('./temp_user_topic_stat/all_dest/topic_feature.txt', sep='\t')
test = pd.merge(test, ut_stat_all, on='uid', how='left')

# 压缩数据
int_feat = ['count_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic']
test[int_feat] = test[int_feat].fillna(0).astype('int32')
for f in ut_stat_feat:
    if f not in int_feat:
        test[f] = test[f].astype('float32')

In [ ]:
# 保存数据
train[ut_stat_feat].to_csv(f'{feature_path}/train_kfold_ut_feature.txt', index=False, sep='\t')
test[ut_stat_feat].to_csv(f'{feature_path}/test_kfold_ut_feature.txt', index=False, sep='\t')

## 新测试集

In [8]:
test2 = pd.read_csv(f'{base_path}/invite_info_evaluate_2_0926.txt', sep='\t', header=None)
test2.columns = ['qid', 'uid', 'dt']
del test2['dt']
logging.info("test2 %s", test2.shape)

[2019-12-17 04:28:17,368] INFO in <ipython-input-8-6d1f561dc942>: test2 (1141718, 2)


In [9]:
ut_stat_all = pd.read_csv('./temp_user_topic_stat/all_dest/topic_feature.txt', sep='\t')
test2 = pd.merge(test2, ut_stat_all, on='uid', how='left')
logging.info("test2 %s", test2.shape)

[2019-12-17 04:28:31,350] INFO in <ipython-input-9-c7c956b507d1>: test2 (1141718, 19)


In [10]:
# 压缩数据
ut_stat_feat = ['count_u_topic', 'mean_u_topic', 'std_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic', 
                'min_uu_sim', 'max_uu_sim','sum_uu_sim', 'mean_uu_sim', 'std_uu_sim', 
                'min_uu_sim_eucl', 'max_uu_sim_eucl', 'sum_uu_sim_eucl', 'mean_uu_sim_eucl', 'std_uu_sim_eucl']
int_feat = ['count_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic']
test2[int_feat] = test2[int_feat].fillna(0).astype('int32')
for f in ut_stat_feat:
    if f not in int_feat:
        test2[f] = test2[f].astype('float32')

In [12]:
test2[ut_stat_feat].to_csv(f'{feature_path}/test_kfold_ut_feature_2.txt', index=False, sep='\t')

# feature 2

In [10]:
# 问题话题与用户在其他时期回答的话题交叉特征
def qu_topic_sim(q_topic, u_topic):
    n_most_common = 20     # 截取回答次数前面的话题
        
    if (q_topic == '-1' or u_topic == '-1'):
        return ([0]*4 + [np.nan]*20)
    
    q_topic_2 = q_topic.split(',')
    u_topic_2 = u_topic.split(',')
    counter = Counter(u_topic_2)          
    most_common_topic = counter.most_common(n_most_common)
    
    u_topic_most_com = []     # 前几个最常回答的话题
    count_list = []           # 这几个常回答的话题对应的计数
    for ut_c in most_common_topic: 
        u_topic_most_com.append(ut_c[0])
        count_list.append(ut_c[1])
    count_norm_list = np.array(list(count_list)) / np.sum(list(count_list))   # 这几个常回答话题对应的计数比例
    
    """
    问题话题与用户话题交互统计
    """
    
    count_ut_in_qt_weighted = 0     # 问题话题在过往回答过的话题里的出现次数
    count_ut_in_qt = 0             # 问题话题与过往回答过的话题的交集个数
    count_norm_ut_in_qt = 0
    # 余弦相似度
    sim_list = []                   # 两两交叉计算相似度
    sim_norm_list = []
    # 欧氏距离
    sim_list_eucl = []                   # 两两交叉计算相似度
    sim_norm_list_eucl = []

    for qt in q_topic_2:
        q_topic_index = int(qt[1:]) - 1   # q_topic 索引
        q_topic_vec = topic_table.iloc[q_topic_index]['vec']
        for ut_c, count, count_norm in zip(most_common_topic, count_list, count_norm_list):  
            u_topic_index = int(ut_c[0][1:]) - 1   # u_topic 索引
            u_topic_vec = topic_table.iloc[u_topic_index]['vec']
            if q_topic_index == u_topic_index:
                count_ut_in_qt_weighted += ut_c[1]
                count_ut_in_qt += 1
                count_norm_ut_in_qt += count_norm
            
            sim = cos_sim(q_topic_vec, u_topic_vec)
            sim_norm = sim*count_norm
            sim_eucl = eucl_sim(q_topic_vec, u_topic_vec)
            sim_norm_eucl = sim_eucl*count_norm
            
            sim_list.append(sim)
            sim_norm_list.append(sim_norm)
            sim_list_eucl.append(sim_eucl)
            sim_norm_list_eucl.append(sim_norm_eucl)
            
    # 出现在问题和回答过的问题的话题，在问题话题中所占比例
    rate_ut_in_qt = count_ut_in_qt / len(q_topic_2)
    
    # 余弦相似度
    min_qu_sim = np.min(sim_list)
    max_qu_sim = np.max(sim_list)
    sum_qu_sim = np.sum(sim_list)
    mean_qu_sim = np.mean(sim_list)
    std_qu_sim = np.std(sim_list)
    min_qu_sim_norm = np.min(sim_norm_list)
    max_qu_sim_norm = np.max(sim_norm_list)
    sum_qu_sim_norm = np.sum(sim_norm_list)
    mean_qu_sim_norm = np.mean(sim_norm_list)
    std_qu_sim_norm = np.std(sim_norm_list)
    # 欧氏距离
    min_qu_sim_eucl = np.min(sim_list_eucl)
    max_qu_sim_eucl = np.max(sim_list_eucl)
    sum_qu_sim_eucl = np.sum(sim_list_eucl)
    mean_qu_sim_eucl = np.mean(sim_list_eucl)
    std_qu_sim_eucl = np.std(sim_list_eucl)
    min_qu_sim_norm_eucl = np.min(sim_norm_list_eucl)
    max_qu_sim_norm_eucl = np.max(sim_norm_list_eucl)
    sum_qu_sim_norm_eucl = np.sum(sim_norm_list_eucl)
    mean_qu_sim_norm_eucl = np.mean(sim_norm_list_eucl)
    std_qu_sim_norm_eucl = np.std(sim_norm_list_eucl)
    
    # 问题话题与用户回答话题的统计
    res = [count_ut_in_qt_weighted, count_ut_in_qt, rate_ut_in_qt, count_norm_ut_in_qt]
    res += [min_qu_sim, max_qu_sim, sum_qu_sim, mean_qu_sim, std_qu_sim]
    res += [min_qu_sim_norm, max_qu_sim_norm, sum_qu_sim_norm, mean_qu_sim_norm, std_qu_sim_norm]
    res += [min_qu_sim_eucl, max_qu_sim_eucl, sum_qu_sim_eucl, mean_qu_sim_eucl, std_qu_sim_eucl]
    res += [min_qu_sim_norm_eucl, max_qu_sim_norm_eucl, sum_qu_sim_norm_eucl, mean_qu_sim_norm_eucl, std_qu_sim_norm_eucl]
    return res

In [14]:
import traceback

processes = 36   # 设置进程数

def cross_qu_topic_feat(df, num, mode):
    try:
        logging.info('start to extract feature.')
        t1 = df.apply(lambda x: qu_topic_sim(x['topic'], x['user_topic_kfold']), axis=1, result_type='expand')
        col_name = ['qu_topic_count_weight', 'qu_topic_count', 'qu_topic_rate', 'qu_topic_count_norm',
                  'min_sim', 'max_sim', 'sum_sim', 'mean_sim', 'std_sim', 'min_sim_norm', 
                  'max_sim_norm', 'sum_sim_norm', 'mean_sim_norm', 'std_sim_norm', 
                  'min_sim_eucl', 'max_sim_eucl', 'sum_sim_eucl', 'mean_sim_eucl', 'std_sim_eucl',
                  'min_sim_norm_eucl', 'max_sim_norm_eucl', 'sum_sim_norm_eucl', 'mean_sim_norm_eucl', 'std_sim_norm_eucl']
        t1.columns = col_name
        logging.info('extracting finish.')
        
        # 压缩数据
        t1 = compress_data(t1)
        t1.to_csv(f'./temp/{mode}_kfold_topic_feature_{num}.txt', index=False, sep='\t')
        logging.info('file %s saving finish.', num)
        del t1
        gc.collect()
    except:
        print(traceback.print_exc())
        
def multi_proc(df, mode):
    import multiprocessing
    pool = multiprocessing.Pool(processes=processes)
    len_data = len(df)
    len_batch = len_data // processes
    for i in range(processes):
        start = i * len_batch
        end = (i+1) * len_batch
        if i == (processes-1):
            end = len_data
        tmp = df[start:end]
        pool.apply_async(cross_qu_topic_feat, (tmp, i, mode))
        
    pool.close()
    pool.join()

In [ ]:
multi_proc(test, 'test')

In [ ]:
multi_proc(train, 'train')

In [ ]:
# 合数据

# test
topic_feat = None
original_path = './temp'
mode = 'test'
for i in range(processes):
    d = pd.read_csv(f'{original_path}/{mode}_kfold_topic_feature_{i}.txt', sep='\t')
    if topic_feat is None:
        topic_feat = d
    else:
        topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
        
logging.info('%s topic feature, shape: %s', mode, topic_feat.shape)

topic_feat = compress_data(topic_feat)
topic_feat.to_csv(f'{feature_path}/{mode}_kfold_topic_feature.txt', index=False, sep='\t')
logging.info('%s topic feature saved.', mode)

In [ ]:
# train
topic_feat = None
original_path = './temp'
mode = 'train'
for i in range(processes):
    d = pd.read_csv(f'{original_path}/{mode}_kfold_topic_feature_{i}.txt', sep='\t')
    if topic_feat is None:
        topic_feat = d
    else:
        topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
logging.info('%s topic feature, shape: %s', mode, topic_feat.shape)

topic_feat = compress_data(topic_feat)
topic_feat.to_csv(f'{feature_path}/{mode}_kfold_topic_feature.txt', index=False, sep='\t')
logging.info('%s topic feature saved.', mode)

## 新测试集

In [11]:
# test2
user_topic = pickle.load(open(f'{base_path}/user_topic.pkl', 'rb'))       # 没有打折
test2['user_topic_kfold'] = test2['uid'].map(user_topic).fillna('-1')    # 实际并不是 kfold

In [12]:
import traceback

processes = 36   # 设置进程数

def cross_qu_topic_feat(df, num, mode):
    try:
        logging.info('start to extract feature.')
        t1 = df.apply(lambda x: qu_topic_sim(x['topic'], x['user_topic_kfold']), axis=1, result_type='expand')
        col_name = ['qu_topic_count_weight', 'qu_topic_count', 'qu_topic_rate', 'qu_topic_count_norm',
                  'min_sim', 'max_sim', 'sum_sim', 'mean_sim', 'std_sim', 'min_sim_norm', 
                  'max_sim_norm', 'sum_sim_norm', 'mean_sim_norm', 'std_sim_norm', 
                  'min_sim_eucl', 'max_sim_eucl', 'sum_sim_eucl', 'mean_sim_eucl', 'std_sim_eucl',
                  'min_sim_norm_eucl', 'max_sim_norm_eucl', 'sum_sim_norm_eucl', 'mean_sim_norm_eucl', 'std_sim_norm_eucl']
        t1.columns = col_name
        logging.info('extracting finish.')
        
        # 压缩数据
        t1 = compress_data(t1)
        t1.to_csv(f'./temp/test2/{mode}_kfold_topic_feature_{num}.txt', index=False, sep='\t')
        logging.info('file %s saving finish.', num)
        del t1
        gc.collect()
    except:
        print(traceback.print_exc())
        
def multi_proc(df, mode):
    import multiprocessing
    pool = multiprocessing.Pool(processes=processes)
    len_data = len(df)
    len_batch = len_data // processes
    for i in range(processes):
        start = i * len_batch
        end = (i+1) * len_batch
        if i == (processes-1):
            end = len_data
        tmp = df[start:end]
        pool.apply_async(cross_qu_topic_feat, (tmp, i, mode))
        
    pool.close()
    pool.join()

In [15]:
multi_proc(test2, 'test')

[2019-12-17 04:30:06,547] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:06,724] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:06,931] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,160] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,320] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,452] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,560] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,675] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,786] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:07,923] INFO in <ipython-input-12-aa12bbe8f0d9>: start to extract feature.
[2019-12-17 04:30:08,247] INFO in <ipython-input-12-aa12bbe8f0d9>: sta

[2019-12-17 07:40:17,973] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:40:18,818] INFO in <ipython-input-12-aa12bbe8f0d9>: file 26 saving finish.
[2019-12-17 07:40:40,969] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:40:41,832] INFO in <ipython-input-12-aa12bbe8f0d9>: file 27 saving finish.
[2019-12-17 07:40:48,570] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:40:49,467] INFO in <ipython-input-12-aa12bbe8f0d9>: file 35 saving finish.
[2019-12-17 07:40:55,408] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:40:56,257] INFO in <ipython-input-12-aa12bbe8f0d9>: file 31 saving finish.
[2019-12-17 07:40:56,426] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:40:57,271] INFO in <ipython-input-12-aa12bbe8f0d9>: file 24 saving finish.
[2019-12-17 07:42:20,624] INFO in <ipython-input-12-aa12bbe8f0d9>: extracting finish.
[2019-12-17 07:42:21,462] INFO in 

In [16]:
# 新数据

# test2
topic_feat = None
original_path = './temp/test2'
mode = 'test'
for i in range(processes):
    d = pd.read_csv(f'{original_path}/{mode}_kfold_topic_feature_{i}.txt', sep='\t')
    if topic_feat is None:
        topic_feat = d
    else:
        topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
        
logging.info('%s topic feature, shape: %s', mode, topic_feat.shape)

topic_feat = compress_data(topic_feat)
topic_feat.to_csv(f'{feature_path}/new{mode}_kfold_topic_feature.txt', index=False, sep='\t')
logging.info('%s topic feature saved.', mode)

[2019-12-17 07:46:38,471] INFO in <ipython-input-16-2d1cb0656709>: test topic feature, shape: (1141718, 24)
[2019-12-17 07:47:09,062] INFO in <ipython-input-16-2d1cb0656709>: test topic feature saved.
